In [1]:
import os
import json
from collections import Counter
from datetime import datetime, timezone, timedelta
from itertools import combinations
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
from tqdm.notebook import tqdm

### Configuration

In [2]:
ID = "037"
SEED = 42
level = "l1"
FEATURE_DIR = Path(f"../../artifacts/features/{ID}")

os.makedirs(FEATURE_DIR, exist_ok=True)

# === Chank処理の個数と処理する番目を確定 ===
CHUNK_SIZE = 5
CHUNK_N = 2

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_rows(500)
pl.Config.set_tbl_cols(500)

print(f"Feature dir created successfully in \n{FEATURE_DIR}")

Feature dir created successfully in 
../../artifacts/features/037


### Utils

In [4]:
def check_info(
    train: pl.DataFrame,
    test: pl.DataFrame
) -> tuple[float, float, float]:
    train_mem = sum(train[col].to_numpy().nbytes for col in train.columns) / 1024**3
    test_mem = sum(test[col].to_numpy().nbytes for col in test.columns) / 1024**3

    print("=== Shape & Memory ===")
    print(f"Train Shape: {train.shape}, Test Shape: {test.shape}")
    print(f"Train Memory: {train_mem:.2f} GB, Test Memory: {test_mem:.2f} GB\n")

    dtype_counts = Counter([str(dt) for dt in train.dtypes])

    n_cat = None
    print("=== DTypes ===")
    for dtype, cnt in dtype_counts.items():
        print(f"{dtype}: {cnt}")
        if dtype == "Categorical":
            n_cat = cnt
    return train_mem, test_mem, n_cat


def downcast(df: pl.DataFrame) -> pl.DataFrame:
    INT32_MIN, INT32_MAX = -2_147_483_648, 2_147_483_647

    df = df.with_columns(pl.col(pl.Float64).cast(pl.Float32))

    # Int64で安全に落とせる列だけ選別
    int64_cols = [c for c, dt in df.schema.items() if dt == pl.Int64]
    safe_cols = []
    for c in int64_cols:
        mn, mx = df[c].min(), df[c].max()
        if mn >= INT32_MIN and mx <= INT32_MAX:
            safe_cols.append(c)

    # 安全な列だけ Int32 に
    if safe_cols:
        df = df.with_columns(pl.col(safe_cols).cast(pl.Int32))
    return df

def add_bin_columns(
    df: pl.DataFrame,
    cols: list[str],
    n_bins: int,
    *,
    strategy: str = "quantile",
    quantile_method: str = "nearest",
    suffix: str = "_bin",
    drop_source: bool = False,
) -> pl.DataFrame:
    edges: dict[str, np.ndarray] = {}

    for c in cols:
        vals = df[c].to_numpy()
        vals = vals[np.isfinite(vals)]
        if vals.size == 0:
            continue

        if strategy == "quantile":
            qs = np.linspace(0.0, 1.0, n_bins + 1)
            qv = np.quantile(vals, qs, method=quantile_method)
            e = np.unique(qv)
        else:
            vmin = float(np.nanmin(vals))
            vmax = float(np.nanmax(vals))
            e = np.linspace(vmin, vmax, n_bins + 1)

        if e.size < 3:
            vmin = np.nanmin(vals)
            vmax = np.nanmax(vals)
            if not np.isfinite(vmin):
                vmin = 0.0
            if not np.isfinite(vmax):
                vmax = vmin + 1.0
            e = np.array([vmin - 1, vmax + 1], dtype="float64")

        edges[c] = e.astype("float64")

    out = df
    for c, e in edges.items():
        bname = f"{c}{suffix}"
        out = out.with_columns(
            pl.col(c)
              .cut(list(e))
              .cast(pl.Categorical)
              .to_physical()
              .rank("dense")
              .alias(bname)
        )
        if drop_source:
            out = out.drop(c)
    return out


def compute_group_stats(
    df: pl.DataFrame,
    key_cols: list[str],
    value_cols: list[str],
    *,
    stats: tuple[str, ...] = ("mean", "std", "min", "max", "median")
    ,
) -> pl.DataFrame:
    stats_dict = {}
    for k in tqdm(key_cols):
        for v in value_cols:
            if v in k:
                continue
            aggs = []
            col_names = []
            if "mean" in stats:
                aggs.append(pl.col(v).mean().alias(f"{v}_mean_by_{k}"))
                col_names.append(f"{v}_mean_by_{k}")
            if "std" in stats:
                aggs.append(pl.col(v).std(ddof=1).alias(f"{v}_std_by_{k}"))
                col_names.append(f"{v}_std_by_{k}")
            if "min" in stats:
                aggs.append(pl.col(v).min().alias(f"{v}_min_by_{k}"))
                col_names.append(f"{v}_min_by_{k}")
            if "max" in stats:
                aggs.append(pl.col(v).max().alias(f"{v}_max_by_{k}"))
                col_names.append(f"{v}_max_by_{k}")
            if "median" in stats:
                aggs.append(pl.col(v).median().alias(f"{v}_median_by_{k}"))
                col_names.append(f"{v}_median_by_{k}")

            grouped_df = (
                df.select([k, v])
                .group_by(k)
                .agg(aggs)
            )
            stats_array = (
                df.join(
                    grouped_df.select(col_names + [k]),
                    on=k,
                    how="left"
                )
                .select(col_names)
                .to_numpy()
                .astype(dtype=np.float32, copy=False)
            )
            for i, c in enumerate(col_names):
                stats_dict[c] = stats_array[:, i]

            del grouped_df, stats_array

    stats_df = pl.DataFrame(stats_dict)

    return stats_df

def drop_low_std_cols(
    df: pl.DataFrame,
    threshold: float = 0,
    ddof: int = 0,
) -> list[str]:
    cols = [c for c, t in df.schema.items() if t not in [pl.Categorical, pl.Utf8]]

    # 安定のため float にキャストしてから std
    std_row = (
        df.select([pl.col(c).cast(pl.Float64).std(ddof=ddof).alias(c) for c in cols])
          .row(0)
    )
    names = cols
    values = std_row
    low_cols = [c for c, s in zip(names, values)
                if (s is None) or (not np.isfinite(s)) or (s <= threshold)]
    print(f"n drop cols: {len(low_cols)}\n{low_cols}")
    return df.drop(low_cols)

### Feature Engineering
- 035派生
- num_df2とcat_dfで1-gram stats_df(mean, std, median, max)
- durationのdigits_df
- 欠損値を含む列をdrop

In [5]:
# === Load Data ===
train = pl.read_csv("../../input/train.csv").drop("id")
test = pl.read_csv("../../input/test.csv").drop("id")
orig = pl.read_parquet("../../input/original.parquet")
orig = orig.with_columns(
    pl.when(pl.col("y") == "yes").then(1)
      .when(pl.col("y") == "no").then(0)
      .otherwise(None)
      .alias("y")
)

y_tr = train["y"].cast(pl.Int8)
y_orig = orig["y"].cast(pl.Int8)
y_merged = pl.concat([y_tr, y_orig], how="vertical")

train = train.drop("y")
orig = orig.drop("y")

CATS = [col for col in train.columns if train[col].dtype == pl.Utf8]
NUMS = [col for col in train.columns if train[col].dtype != pl.Utf8]
print(f"NUMS: {len(NUMS)}\n{NUMS}")
print(f"\nCATS: {len(CATS)}\n{CATS}")

NUMS: 7
['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

CATS: 9
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']


In [6]:
# === 全データを結合 ===
all_data = pl.concat([train, test], how="vertical")
cat_exprs = [
    (pl.col(c)
    .cast(pl.Categorical)
    .rank("dense") - 1)
    .cast(pl.Int32)
    .cast(pl.Utf8)
    .cast(pl.Categorical)
    .alias(c)
    for c in CATS
]
all_data = all_data.with_columns(cat_exprs)
num_df = all_data.select(NUMS)
cat_df = all_data.select(CATS)

In [7]:
NUMS2CATS = [f"{c}2" for c in NUMS]
num2cat_exprs = [
    pl.col(c)
    .cast(pl.Utf8)
    .cast(pl.Categorical)
    .to_physical()
    .rank("dense")
    .cast(pl.Int32).alias(f"{c}2")
    for c in NUMS
]
num_df2 = num_df.select(num2cat_exprs)
print(f"num_df2: {len(num_df2.columns)}\n{num_df2.columns}")

num_df2: 7
['age2', 'balance2', 'day2', 'duration2', 'campaign2', 'pdays2', 'previous2']


In [8]:
# Grouped Dfを作成
stats_df = compute_group_stats(
    pl.concat([cat_df, num_df2, num_df], how="horizontal"),
    CATS + NUMS2CATS,
    NUMS,
    stats=("mean", "std", "max", "median")
)
print(f"Created {len(stats_df.columns)} new columns")

  0%|          | 0/16 [00:00<?, ?it/s]

Created 416 new columns


In [9]:
# Durationのそれぞれの桁の数
duration_df = num_df.select(pl.col("duration"))

exprs = []
for k in (0, 1, 2, 3):
    exprs.append(
        ((pl.col("duration") // (10**k)) % 10)
        .cast(pl.Int32)
        .alias(f"duration_digitL{k}")
    )

duration_df = duration_df.with_columns(exprs).drop("duration")

In [10]:
# Merge Data
all_data = pl.concat([num_df, cat_df, stats_df, duration_df], how="horizontal")

In [11]:
# === drop low std cols ===
all_data = drop_low_std_cols(all_data)

# === row_id を追加 ===
all_data = all_data.with_row_index("row_id")

# === Downcast ===
all_data = downcast(all_data)

# === データを分割 ===
tr_df = all_data[:len(train)]
test_df = all_data[len(train):len(train)+len(test)]

# === targetを追加 ===
tr_df = tr_df.with_columns(y_tr.alias("target"))

n drop cols: 76
['day_max_by_job', 'pdays_median_by_job', 'previous_median_by_job', 'day_max_by_marital', 'campaign_median_by_marital', 'pdays_median_by_marital', 'previous_median_by_marital', 'age_max_by_education', 'day_max_by_education', 'campaign_median_by_education', 'pdays_median_by_education', 'previous_median_by_education', 'age_median_by_default', 'day_max_by_default', 'campaign_median_by_default', 'pdays_median_by_default', 'previous_median_by_default', 'day_max_by_housing', 'campaign_median_by_housing', 'pdays_max_by_housing', 'pdays_median_by_housing', 'previous_median_by_housing', 'age_median_by_loan', 'day_max_by_loan', 'day_median_by_loan', 'campaign_median_by_loan', 'pdays_median_by_loan', 'previous_median_by_loan', 'day_max_by_contact', 'campaign_median_by_contact', 'pdays_max_by_contact', 'pdays_median_by_contact', 'previous_median_by_contact', 'day_max_by_month', 'pdays_median_by_month', 'previous_median_by_month', 'day_max_by_poutcome', 'balance_std_by_age2', 'day_s

### Add Fold Col

In [12]:
# Add Fold Col
folds_path = "../../artifacts/folds/folds.parquet"
pairs = [
    ("skf/k=5/s=42@train", "5fold-s42")
]
cfgs = [c for c, _ in pairs]
rename_map = {c: n for c, n in pairs}

# folds をまとめて読み → ワイド化（cfg列を列見出しに）→ 列名をfold_nameにリネーム
folds_wide = (
    pl.scan_parquet(folds_path)
      .filter(pl.col("cfg").is_in(cfgs))
      .unique(subset=["row_id", "cfg"], keep="last")
      .select(["row_id", "cfg", "fold"])
      .collect(engine="streaming")
      .pivot(values="fold", index="row_id", columns="cfg", aggregate_function="first")
      .rename(rename_map)
      .with_columns(pl.col("row_id").cast(pl.Int32))
      .with_columns([pl.all().exclude("row_id").cast(pl.Int8)])  # 型を軽く
)

# tr_df が DataFrame の場合
tr_df = tr_df.join(folds_wide, on="row_id", how="left")

/tmp/ipykernel_32018/4243328503.py:16: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(values="fold", index="row_id", columns="cfg", aggregate_function="first")


In [13]:
# === 特徴量エンジニアリング後の情報 ===
train_mem, test_mem, n_cat = check_info(tr_df, test_df)

=== Shape & Memory ===
Train Shape: (750000, 363), Test Shape: (250000, 361)
Train Memory: 1.04 GB, Test Memory: 0.34 GB

=== DTypes ===
UInt32: 1
Int32: 11
Categorical(ordering='physical'): 9
Float32: 340
Int8: 2


In [14]:
# === Save Data ===
tr_path = FEATURE_DIR / "train.parquet"
test_path = FEATURE_DIR / "test.parquet"

tr_df.write_parquet(tr_path)
test_df.write_parquet(test_path)

print(f"tr_df saved successfully to {tr_path}")
print(f"test_df saved successfully to {test_path}")

tr_df saved successfully to ../../artifacts/features/037/train.parquet
test_df saved successfully to ../../artifacts/features/037/test.parquet


### Save Meta data

In [15]:
JST = timezone(timedelta(hours=9))
meta = {
    "data_id": ID,
    "train_paths": [str(FEATURE_DIR / "train.parquet")],
    "test_paths": [str(FEATURE_DIR / "test.parquet")],
    "level": level,
    "cat_cols": CATS,
    "created_at": datetime.now(JST).isoformat(),
    "train_shape": [tr_df.height, tr_df.width],
    "test_shape": [test_df.height, test_df.width],
    "memory": {
        "train": train_mem,
        "test": test_mem
    },
    "fold_column": pairs,
    "cat_n": n_cat if n_cat else None
}

with open(f"{FEATURE_DIR}/meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)